# Decision Tree
包含两个元素：
1. decision
2. subNode


1. **初始化条件**：`DTNode`对象在初始化时必须接受一个`decision`参数，这个参数有两种可能的形式：
   - 一个函数：它接收一个对象（通常是一个特征向量），并指示应该跟随哪个子节点（当该对象是决策节点时）。
   - 一个值：代表分类或回归的结果（当对象是叶节点时）。

2. **子节点管理**：`DTNode`对象必须有一个名为`children`的属性，这个属性被设置为一个数据结构，它映射决策函数的输出到一个特定的子节点。我们假设决策函数的输出是一个索引，指向一个列表。

3. **预测方法**：`DTNode`还必须有一个名为`predict`的方法，它接收一个输入对象（例如，一个特征向量），并返回该输入在决策树中的结果。

**提示**：`predict`方法是递归的。你可以使用内置函数`callable`来测试一个Python对象是否可调用（在这种情况下是一个函数）。


## Class

### Function as input

In [5]:
class MyClass:
    # def __init__(self, func):
    def init(self, func):
        self.func = func  # 将传入的函数赋值给一个实例变量
    # arguments ; keyword arguments
    
    def call_func(self, *args, **kwargs):
        # 调用存储的函数，并传递任何参数
        return self.func(*args, **kwargs)
    
def greet(name):
    return f"Hello, {name}!"

# 创建 MyClass 的实例，将 greet 函数作为参数传递
my_instance = MyClass()
my_instance.init(greet)

# 通过 MyClass 实例调用传入的 greet 函数
print(my_instance.call_func("World"))


Hello, World!


### Class as Input


In [6]:
class ConsumerClass:
    def __init__(self, class_input):
        self.class_input = class_input  # 将传入的类赋值给一个实例变量
    
    def create_instance(self, *args, **kwargs):
        # 使用传入的类创建一个实例，并传递任何初始化参数
        return self.class_input(*args, **kwargs)
    
    
class Greeter:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f"Hello, {self.name}!"

# 创建 ConsumerClass 的实例，将 Greeter 类作为参数传递
consumer_instance = ConsumerClass(Greeter)

# 通过 ConsumerClass 实例创建 Greeter 的一个实例
greeter_instance = consumer_instance.create_instance("World")

# 调用 greeter_instance 的 greet 方法
print(greeter_instance.greet())


Hello, World!


# Q 1


In [9]:
class DTNode:
    def __init__(self, decision):
        self.decision = decision
        self.children = []

    def predict(self, input_obj):
        if callable(self.decision):
            print("self.decision callable")
            
            child_index = self.decision(input_obj)
            print("child_index " , child_index)
            print("self_index " , len(self.children))
            
            if child_index < len(self.children):
                return self.children[child_index].predict(input_obj)
            else:
                raise ValueError("Error")
        else:
            print("The decision is ",self.decision)
            return self.decision

# ==================== TEST CASE ====================

yes_node = DTNode("Yes")
no_node = DTNode("No")
tree_root = DTNode(lambda x: 0 if x[2] < 4 else 1)
tree_root.children = [yes_node, no_node]

# print(tree_root.predict((False, 'Red', 3.5)))
print(tree_root.predict((False, 'Green', 6.1)))



self.decision callable
child_index  1
self_index  2
The decision is  No
No


# q2

In [16]:
class DTNode:
    def __init__(self, decision):
        self.decision = decision
        self.children = []

    def predict(self, input_obj):
        if callable(self.decision):
            print("self.decision callable")
            
            child_index = self.decision(input_obj)
            print("child_index " , child_index)
            print("self_index " , len(self.children))
            
            if child_index < len(self.children):
                return self.children[child_index].predict(input_obj)
            else:
                raise ValueError("Error")
        else:
            print("The decision is ",self.decision)
            return self.decision
        
    def leaves(self):
        if not callable(self.decision):
            return 1
        else:
            return sum(child.leaves() for child in self.children)
            # print("more")

# ==================== TEST CASE ====================
	
t = DTNode(True)
f = DTNode(False)
n = DTNode(lambda v: 0 if not v else 1)
n.children = [t, f]
print(n.leaves())




more
None


# Q3


In [31]:
def partition_by_feature_value(dataset, feature_index):
    partitions = {}
    for x, y in dataset:
        key = x[feature_index]
        print("Key: ",key)
        
        print("Partitions: ")
        for _ in partitions:
            print(_)

        if key not in partitions:
            partitions[key] = []
        partitions[key].append((x, y))
    
    # 将分区映射转换为列表，以便通过索引访问
    partition_list = list(partitions.values())
    
    # 定义分隔器函数
    def separator(feature_vector):
        return list(partitions.keys()).index(feature_vector[feature_index])
    
    return separator, partition_list



#### Test case 1

In [18]:
# ------------- TEST 
from pprint import pprint
dataset = [
  ((True, True), False),
  ((True, False), True),
  ((False, True), True),
  ((False, False), False),
]
f, p = partition_by_feature_value(dataset,  0)
pprint(sorted(sorted(partition) for partition in p))

partition_index = f((True, True))
# Everything in the "True" partition for feature 0 is true
print(all(x[0]==True for x,c in p[partition_index]))
partition_index = f((False, True))
# Everything in the "False" partition for feature 0 is false
print(all(x[0]==False for x,c in p[partition_index]))

[[((False, False), False), ((False, True), True)],
 [((True, False), True), ((True, True), False)]]
True
True


#### Test case 2

In [46]:
# ------ test 2

from pprint import pprint
dataset = [
  (("a", "x", 2), False),
  (("b", "x", 2), False),
  (("a", "y", 5), True),
  (("c", "y", 5), False),
]
f, p = partition_by_feature_value(dataset, 0)
print("separator: ")
print(f)
print("\nPartition_List")
print(p)
pprint(sorted(sorted(partition) for partition in p))
partition_index = f(("a", "y", 5))
# everything in the "y" partition for feature 1 has a y
print(all(x[0]=="a" for x, c in p[partition_index]))

Key:  a
Partitions: 
Key:  b
Partitions: 
a
Key:  a
Partitions: 
a
b
Key:  c
Partitions: 
a
b
separator: 
<function partition_by_feature_value.<locals>.separator at 0x12213d4e0>

Partition_List
[[(('a', 'x', 2), False), (('a', 'y', 5), True)], [(('b', 'x', 2), False)], [(('c', 'y', 5), False)]]
[[(('a', 'x', 2), False), (('a', 'y', 5), True)],
 [(('b', 'x', 2), False)],
 [(('c', 'y', 5), False)]]
True


# Q4 


In [47]:
from math import log2

def calculate_proportions(dataset):
    counts = {}
    for _, classification in dataset:
        counts[classification] = counts.get(classification, 0) + 1
    total = len(dataset)
    proportions = {k: v / total for k, v in counts.items()}
    return proportions.values()

def misclassification(dataset):
    proportions = calculate_proportions(dataset)
    return 1 - max(proportions)

def gini(dataset):
    proportions = calculate_proportions(dataset)
    return sum(p * (1 - p) for p in proportions)

def entropy(dataset):
    proportions = calculate_proportions(dataset)
    return -sum(p * log2(p) for p in proportions if p > 0)

# 用于测试的数据集
data = [
    ((False, False), False),
    ((False, True), True),
    ((True, False), True),
    ((True, True), False)
]

# 执行测试并打印结果
data = [
    ((False, False), False),
    ((False, True), True),
    ((True, False), True),
    ((True, True), False)
]
print("{:.4f}".format(misclassification(data)))
print("{:.4f}".format(gini(data)))
print("{:.4f}".format(entropy(data)))


0.5000
0.5000
1.0000


# Q5

In [58]:
class DTNode:
    def __init__(self, decision=None, label=None):
        self.decision = decision  # Can be a callable (for decision nodes) or None (for leaf nodes)
        self.label = label  # Used for leaf nodes
        self.children = {}  # Maps feature values to DTNode instances

    def predict(self, input_obj):
        if self.decision is None:
            return self.label
        else:
            feature_value = self.decision(input_obj)
            if feature_value in self.children:
                return self.children[feature_value].predict(input_obj)
            else:
                return None  # Or some default prediction

def partition_by_feature_value(dataset, feature_index):
    partitions = {}
    for x, y in dataset:
        key = x[feature_index]
        if key not in partitions:
            partitions[key] = []
        partitions[key].append((x, y))

    partition_list = list(partitions.values())

    def separator(feature_vector):
        return list(partitions.keys()).index(feature_vector[feature_index])

    return separator, partition_list

def misclassification(dataset):
    proportions = calculate_proportions(dataset)
    return 1 - max(proportions)
# def misclassification(dataset):
#     proportions = calculate_proportions(dataset)
#     return 1 - max(proportions.values())
def calculate_proportions(dataset):
    """Calculate the proportions of each classification in the dataset."""
    counts = {}
    for _, classification in dataset:
        counts[classification] = counts.get(classification, 0) + 1
    total = len(dataset)
    return {k: v / total for k, v in counts.items()}


def train_tree(dataset, criterion, depth=0):
    # Base case: if all examples have the same classification, return a leaf node
    if len(set(y for _, y in dataset)) == 1:
        return DTNode(label=dataset[0][1])

    # Choose the best feature to split on
    num_features = len(dataset[0][0])
    best_feature, best_impurity = None, float('inf')
    for feature_index in range(num_features):
        _, partitions = partition_by_feature_value(dataset, feature_index)
        impurity = sum(criterion(part) for part in partitions)
        if impurity < best_impurity:
            best_feature, best_impurity = feature_index, impurity

    if best_feature is None:
        # Return a leaf node with the most common label if no improvement can be made
        labels = [y for _, y in dataset]
        most_common_label = max(set(labels), key=labels.count)
        return DTNode(label=most_common_label)

    # Partition the dataset by the best feature and create child nodes
    separator, partitions = partition_by_feature_value(dataset, best_feature)
    node = DTNode(decision=lambda x: x[best_feature])
    for i, partition in enumerate(partitions):
        child = train_tree(partition, criterion, depth + 1)
        node.children[list(separator.keys())[i]] = child

    return node


In [60]:
from math import log2

class DTNode:
    def __init__(self, decision=None, label=None):
        self.decision = decision  # Can be a callable (for decision nodes) or None (for leaf nodes)
        self.label = label  # Used for leaf nodes
        self.children = {}  # Maps feature values to DTNode instances

    def predict(self, input_obj):
        if self.decision is None:
            return self.label
        else:
            feature_value = self.decision(input_obj)
            if feature_value in self.children:
                return self.children[feature_value].predict(input_obj)
            else:
                return None  # Or some default prediction

def calculate_proportions(dataset):
    counts = {}
    for _, classification in dataset:
        counts[classification] = counts.get(classification, 0) + 1
    total = len(dataset)
    return {k: v / total for k, v in counts.items()}

def misclassification(dataset):
    proportions = calculate_proportions(dataset)
    return 1 - max(proportions.values())

def partition_by_feature_value(dataset, feature_index):
    partitions = {}
    for x, y in dataset:
        key = x[feature_index]
        if key not in partitions:
            partitions[key] = []
        partitions[key].append((x, y))

    partition_list = list(partitions.values())

    def separator(feature_vector):
        return list(partitions.keys()).index(feature_vector[feature_index])

    return separator, partition_list

def train_tree(dataset, criterion, depth=0):
    if len(set(y for _, y in dataset)) == 1:
        return DTNode(label=dataset[0][1])

    num_features = len(dataset[0][0])
    best_feature, best_impurity = None, float('inf')
    for feature_index in range(num_features):
        _, partitions = partition_by_feature_value(dataset, feature_index)
        impurity = sum(criterion(part) for part in partitions)
        if impurity < best_impurity:
            best_feature, best_impurity = feature_index, impurity

    if best_feature is None:
        labels = [y for _, y in dataset]
        most_common_label = max(set(labels), key=labels.count)
        return DTNode(label=most_common_label)

    separator, partitions = partition_by_feature_value(dataset, best_feature)
    node = DTNode(decision=lambda x: x[best_feature])
    for i, partition in enumerate(partitions):
        child = train_tree(partition, criterion, depth + 1)
        node.children[list(partitions.keys())[i]] = child

    return node

# Example Test
dataset = [
  ((True, True), False),
  ((True, False), True),
  ((False, True), True),
  ((False, False), False)
]
t = train_tree(dataset, misclassification)
print(t.predict((True, False)))  # Expected: True
print(t.predict((False, False)))  # Expected: False


AttributeError: 'list' object has no attribute 'keys'

In [62]:
dataset = [
  ((True, True), False),
  ((True, False), True),
  ((False, True), True),
  ((False, False), False)
]
t = train_tree(dataset, misclassification)
print(t.predict((True, False)))
print(t.predict((False, False)))

True
False


In [2]:
def partition_by_feature_value(dataset, feature_index):
    partitions = {}
    for features, label in dataset:
        feature_value = features[feature_index]
        if feature_value not in partitions:
            partitions[feature_value] = []
        partitions[feature_value].append((features, label))
    return partitions

class DTNode:
    def __init__(self, decision=None, label=None):
        self.decision = decision  # Function for decision nodes
        self.label = label  # Label for leaf nodes
        self.children = {}  # Children nodes

    def predict(self, input_obj):
        if self.decision is None:  # Leaf node
            return self.label
        else:  # Decision node
            feature_value = self.decision(input_obj)
            if feature_value in self.children:
                return self.children[feature_value].predict(input_obj)
            return None

def train_tree(dataset, criterion, feature_index=0, depth=0):
    # Base case: if all examples have the same classification or no features left
    classifications = [y for _, y in dataset]
    if len(set(classifications)) == 1:
        return DTNode(label=classifications[0])

    # Find the best feature to split on
    best_feature = feature_index  # Simplified; in practice, you might evaluate each feature

    partitions = partition_by_feature_value(dataset, best_feature)

    # Decision node based on the best feature; adjust if evaluating best feature
    node = DTNode(decision=lambda x: x[best_feature])
    for feature_value, partition in partitions.items():
        if len(partition) > 0:
            # Recursively build the tree
            child = train_tree(partition, criterion, feature_index + 1, depth + 1)
            node.children[feature_value] = child
        else:
            # Handle empty partition if necessary
            pass

    return node
def misclassification(dataset):
    proportions = calculate_proportions(dataset)
    return 1 - max(proportions.values())

dataset = [
  ((True, True), False),
  ((True, False), True),
  ((False, True), True),
  ((False, False), False)
]
t = train_tree(dataset, misclassification)
print(t.predict((True, False)))
print(t.predict((False, False)))


True
False
